In [1]:
#imports
import os
import psycopg2
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
def get_data(region='NA1',game_mode='ARAM',patch='14.13'):
    conn = psycopg2.connect(
        database = os.getenv('DB_NAME'),
        host = os.getenv('DB_HOST'),
        user = os.getenv('DB_USER'),
        password = os.getenv('DB_PASSWORD'),
        port = os.getenv('5432')
    )

    cursor = conn.cursor()
    patch = patch+'%'

    try:
        cursor.execute("SELECT * FROM match_data WHERE region = %s AND game_mode = %s AND patch LIKE %s", (region, game_mode,patch))
        rows = cursor.fetchall()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

    return rows

In [18]:
def main(region,game_mode,patch):
    data = get_data(region=region,game_mode=game_mode,patch=patch)
    for row in data:
        print(row)
        break

main('NA1','ARAM','14.13')

(1, 'NA1', 'ARAM', '14.13.596.7996', ['Rell', 'Velkoz', 'Samira', 'TwistedFate', 'Bard'], ['Tristana', 'Lux', 'Neeko', 'Lulu', 'Belveth'], 0)
